## 0. 라이브러리 다운로드

In [2]:
# !pip install torch transformers

   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 1.6/203.0 MB 7.6 MB/s eta 0:00:27
    --------------------------------------- 3.4/203.0 MB 7.7 MB/s eta 0:00:26
    --------------------------------------- 4.2/203.0 MB 7.6 MB/s eta 0:00:27
   - -------------------------------------- 5.5/203.0 MB 6.5 MB/s eta 0:00:31
   -- ------------------------------------- 11.8/203.0 MB 11.4 MB/s eta 0:00:17
   --- ------------------------------------ 19.4/203.0 MB 15.5 MB/s eta 0:00:12
   ----- ---------------------------------- 29.1/203.0 MB 19.8 MB/s eta 0:00:09
   ------ --------------------------------- 34.1/203.0 MB 20.6 MB/s eta 0:00:09
   -------- ------------------------------- 42.7/203.0 MB 22.8 MB/s eta 0:00:08
   ---------- ----------------------------- 54.3/203.0 MB 26.0 MB/s eta 0:00:06
   ------------ --------------------------- 61.3/203.0 MB 26.8 MB/s eta 0:00:06
   -------------- ------------------------- 73.1/203.0 MB

In [5]:
# !pip install accelerate

In [ ]:
# !python -m pip install librosa soundfile langchain sentence-transformers

In [ ]:
# !pip install numpy==1.23.4

     ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
     ------------- -------------------------- 3.7/10.7 MB 18.2 MB/s eta 0:00:01
     --------------------------------------- 10.7/10.7 MB 27.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "C:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "C:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\si

In [19]:
# !pip install langchain sentence-transformers

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 23.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 36.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 58.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 50.3 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using 

## 1. 모듈 불러오기

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import librosa

c:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.1. 모델 불러오기 및 파이프라인 설정

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype,
    # low_cpu_mem_usage=True, 
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cpu


In [3]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")

In [10]:
dataset

Dataset({
    features: ['audio'],
    num_rows: 1
})

In [11]:
dataset[0]

{'audio': {'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
         0.0005188 ]),
  'sampling_rate': 16000}}

In [6]:
sample = dataset[0]["audio"]

In [9]:
type(sample)

dict

In [7]:
result = pipe(sample)
print(result["text"])

c:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\whisper\generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


ValueError: You have passed more than 3000 mel input features (> 30 seconds) which automatically enables long-form generation which requires the model to predict timestamp tokens. Please either pass `return_timestamps=True` or make sure to pass no more than 3000 mel input features.

In [14]:
!pip install pydub

In [15]:
from pydub import AudioSegment

# 오디오 파일 로드
# audio = AudioSegment.from_file("path_to_audio_file.wav")
audio = sample

# 30초 단위로 분할
chunk_length_ms = 30 * 1000  # 30초
chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

# 각 분할 저장
for i, chunk in enumerate(chunks):
    chunk.export(f"chunk_{i}.wav", format="wav")

ModuleNotFoundError: No module named 'pydub'